In [1]:
import pickle
import os

In [2]:
os.chdir('../CSE508_Winter2023_Dataset/InvertedIndex')

In [3]:
#load the invertedindex data
with open('invertedindex.pickle', 'rb') as handle:
    invertedindex = pickle.load(handle)

def get_posting_list(term):
    if term in invertedindex:
        return invertedindex[term][0]
    else:
        return []

def get_posting_listsize(term):
    if term in invertedindex:
        return invertedindex[term][1]

os.chdir('../CSE508_Winter2023_Dataset')
all_docs = os.listdir()

In [12]:
# and query and number of minimum iterations to find and of two terms
def and_query(term1, term2):
    posting_list1 = get_posting_list(term1)
    posting_list2 = get_posting_list(term2)
    posting_list1_size = get_posting_listsize(term1)
    posting_list2_size = get_posting_listsize(term2)
    return and_query_helper(posting_list1, posting_list1_size, posting_list2, posting_list2_size)

def and_query_helper(posting_list1, size1, posting_list2, size2):
    result = []
    count = 0
    i = 0
    j = 0
    while i < size1 and j < size2:
        count += 1
        if posting_list1[i] == posting_list2[j]:
            result.append(posting_list1[i])
            i += 1
            j += 1
        elif posting_list1[i] < posting_list2[j]:
            i += 1
        else:
            j += 1
    return result, count


def and_not_query(term1, term2):
    posting_list1 = get_posting_list(term1)
    posting_list2 = get_posting_list(term2)
    posting_list1_size = get_posting_listsize(term1)
    posting_list2_size = get_posting_listsize(term2)
    return and_not_query_helper(posting_list1, posting_list1_size, posting_list2, posting_list2_size)


def and_not_query_helper(posting_list1, size1, posting_list2, size2):
    result = []
    count = 0
    i = 0
    j = 0
    while i < size1 and j < size2:
        count += 1
        if posting_list1[i] == posting_list2[j]:
            i += 1
            j += 1
        elif posting_list1[i] < posting_list2[j]:
            result.append(posting_list1[i])
            i += 1
        else:
            j += 1
    while i < size1:
        count+=1
        result.append(posting_list1[i])
        i += 1
    return result, count

def or_query(term1, term2):
    posting_list1 = get_posting_list(term1)
    posting_list2 = get_posting_list(term2)
    posting_list1_size = get_posting_listsize(term1)
    posting_list2_size = get_posting_listsize(term2)
    return or_query_helper(posting_list1, posting_list1_size, posting_list2, posting_list2_size)



def or_query_helper(posting_list1, size1, posting_list2, size2):
    result = []
    count = 0
    i = 0
    j = 0
    while i < size1 and j < size2:
        count += 1
        if posting_list1[i] == posting_list2[j]:
            result.append(posting_list1[i])
            i += 1
            j += 1
        elif posting_list1[i] < posting_list2[j]:
            result.append(posting_list1[i])
            i += 1
        else:
            result.append(posting_list2[j])
            j += 1
    if i < size1:
        count += 1
        result.extend(posting_list1[i::])
    if j < size2:
        count += 1
        result.extend(posting_list2[j::])
    return result, count




def or_not_query_helper(posting_list1, size1, posting_list2, size2):
    result = []
    count = 0
    i = 0
    j = 0
    
    while i < size1 and j < size2:
        count += 1
        if posting_list1[i] == posting_list2[j]:
            i += 1
            j += 1
        elif posting_list1[i] < posting_list2[j]:
            result.append(posting_list1[i])
            i += 1
        else:
            j += 1
    while i < size1:
        count += 1
        result.append(posting_list1[i])
        i += 1
    return result, count

In [14]:
print(or_query('slab', 'subjected'))

(['cranfield0005', 'cranfield0006', 'cranfield0029', 'cranfield0031', 'cranfield0051', 'cranfield0066', 'cranfield0090', 'cranfield0091', 'cranfield0144', 'cranfield0229', 'cranfield0274', 'cranfield0349', 'cranfield0386', 'cranfield0391', 'cranfield0395', 'cranfield0419', 'cranfield0454', 'cranfield0485', 'cranfield0509', 'cranfield0533', 'cranfield0579', 'cranfield0582', 'cranfield0587', 'cranfield0596', 'cranfield0625', 'cranfield0627', 'cranfield0640', 'cranfield0644', 'cranfield0707', 'cranfield0720', 'cranfield0721', 'cranfield0723', 'cranfield0726', 'cranfield0729', 'cranfield0735', 'cranfield0743', 'cranfield0779', 'cranfield0820', 'cranfield0822', 'cranfield0827', 'cranfield0836', 'cranfield0837', 'cranfield0838', 'cranfield0845', 'cranfield0846', 'cranfield0863', 'cranfield0868', 'cranfield0884', 'cranfield0887', 'cranfield0891', 'cranfield0938', 'cranfield0952', 'cranfield0957', 'cranfield0982', 'cranfield1022', 'cranfield1031', 'cranfield1037', 'cranfield1043', 'cranfield10

In [15]:
lst = ['cranfield0005', 'cranfield0006', 'cranfield0029', 'cranfield0031', 'cranfield0051', 'cranfield0066', 'cranfield0090', 'cranfield0091', 'cranfield0144', 'cranfield0229', 'cranfield0274', 'cranfield0349', 'cranfield0386', 'cranfield0391', 'cranfield0395', 'cranfield0419', 'cranfield0454', 'cranfield0485', 'cranfield0509', 'cranfield0533', 'cranfield0579', 'cranfield0582', 'cranfield0587', 'cranfield0596', 'cranfield0625', 'cranfield0627', 'cranfield0640', 'cranfield0644', 'cranfield0707', 'cranfield0720', 'cranfield0721', 'cranfield0723', 'cranfield0726', 'cranfield0729', 'cranfield0735', 'cranfield0743', 'cranfield0779', 'cranfield0820', 'cranfield0822', 'cranfield0827', 'cranfield0836', 'cranfield0837', 'cranfield0838', 'cranfield0845', 'cranfield0846', 'cranfield0863', 'cranfield0868', 'cranfield0884', 'cranfield0887', 'cranfield0891', 'cranfield0938', 'cranfield0952', 'cranfield0957', 'cranfield0982', 'cranfield1022', 'cranfield1031', 'cranfield1037', 'cranfield1043', 'cranfield1055', 'cranfield1056', 'cranfield1058', 'cranfield1060', 'cranfield1068', 'cranfield1125', 'cranfield1132', 'cranfield1134', 'cranfield1145', 'cranfield1178', 'cranfield1279', 'cranfield1294', 'cranfield1361', 'cranfield1363', 'cranfield1399']
print(len(lst))

73


In [17]:
print(len(set(lst)))

73


In [ ]:
#now input and output